In [111]:
import itertools
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import onnxruntime as rt

In [130]:
# constants and parameters

data_path = '../data/300k_dataset.csv'
data_path_train = '../data/investigation_train_large_checked.csv'
good_model_path = '../model/good_model.onnx'
biased_model_path = '../model/biased_model.onnx'
features = ['persoon_geslacht_vrouw', 'persoonlijke_eigenschappen_taaleis_voldaan']

In [127]:
# functions

def evaluate(data, predictions, features):
    # Add the predictions to the original data
    data['predictions'] = predictions

    # Generate a combination column representing feature combinations
    feature_combinations = list(itertools.product(*[data[feature].unique() for feature in features]))
    combination_to_label = {combination: chr(65 + i) for i, combination in enumerate(feature_combinations)}
    data['combination'] = data[features].apply(lambda row: combination_to_label[tuple(row)], axis=1)

    # Use crosstab to create a contingency table
    contingency_table = pd.crosstab(data['combination'], data['predictions'])

    # Print the contingency table
    print("Contingency Table:")
    print(contingency_table)

    # Perform the chi-square test on the contingency table
    chi_square, p_value, _, _ = chi2_contingency(contingency_table)

    # Determine if the result is significant
    test_result = p_value < 0.05

    # Return the chi-square statistic, p-value, and test result
    return chi_square, p_value, test_result

# Combinational test function
def combinational_test(data_path: str, model_path: str, features: list):
    # Load the data
    data = pd.read_csv(data_path).drop(['checked', 'Ja', 'Nee'], axis=1).astype(np.float32)

    # Load the ONNX model
    session = rt.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name

    # Run the model to get predictions for all data
    predictions = session.run(None, {input_name: data.values.astype(np.float32)})[0]
    print(f"the shape of prediction is: {predictions.shape}")
    print(f"the type of prediction is: {type(predictions)}")

    # Evaluate the result using the updated evaluate function
    chi_square, p_value, test_result = evaluate(data, predictions, features)

    # Return the evaluation results
    return chi_square, p_value, test_result


In [131]:
res = combinational_test(data_path, good_model_path, features)
print(res)

the shape of prediction is: (300000,)
the type of prediction is: <class 'numpy.ndarray'>
Contingency Table:
predictions      0     1
combination             
A            59502  6048
B            77642  3596
C             7415   860
D            50493  4544
E            81068  3079
F             5191   562
(np.float64(3167.667912856034), np.float64(0.0), np.True_)


In [132]:
res = combinational_test(data_path, biased_model_path, features)
print(res)

the shape of prediction is: (300000,)
the type of prediction is: <class 'numpy.ndarray'>
Contingency Table:
predictions      0      1
combination              
A            52209  13341
B            64482  16756
C             6692   1583
D            55037      0
E            84147      0
F             5753      0
(np.float64(33125.87837821453), np.float64(0.0), np.True_)
